In [1]:
import pandas as pd

In [2]:
mirna = pd.read_csv('../data/METABRIC/_ega-box-04_Agilent_ncRNA_60k_normalised_miRNA_expression.csv', sep='\t')
mirna.shape

(853, 1448)

In [3]:
mirna.head(2)

,ProbeID,miRNA,MB-2718,MB-2616,MB-2634,MB-2643,MB-2613,MB-2629,MB-2645,MB-2626,...,MB-0619,MB-0171,MB-0310,MB-0489,MB-0621,MB-0614,MB-0372,MB-0335,MB-0407,MB-0374
0,A_25_P00013119,hsa-let-7b*,6.292,6.445,6.406,6.333,6.281,6.410,6.386,6.319,...,6.209,6.526,6.344,6.587,6.304,6.324,6.325,6.322,6.301,6.411
1,A_25_P00011980,hsa-let-7d,11.813,10.380,10.028,10.624,9.235,9.818,10.675,10.522,...,10.301,9.944,9.006,9.373,10.316,9.773,9.808,8.085,10.171,10.558


In [4]:
# clean missing more 20
max(mirna.isnull().sum(axis=0).tolist())


0

In [5]:
max(mirna.isnull().sum(axis=1).tolist())


0

In [6]:
b = list()
for c in mirna.columns:
    missing = mirna[c].nonzero()
    if 853 - len(missing[0])   >= 1:
#        del exp[c]
        print(c)

In [4]:
meta_index_mirna = mirna.columns
meta_index_mirna= meta_index_mirna[2:]
tops = ['miRNA']
tops.extend( list(meta_index_mirna))
mirna = mirna[tops]

In [5]:
mirna.shape

(853, 1447)

In [6]:
(mirna.shape[0] /100)* 20

170.6

In [7]:
for c in mirna.columns:
    missing = mirna[c].isnull().sum()
    if missing >= 171:
        del mirna[c]
        print(c)

In [8]:
mirna.shape

(853, 1447)

In [9]:
(1447 / 100 )*20

289.40000000000003

In [10]:
#Find is a feature is missing more than 20%
missing = mirna.isnull().sum(axis=1).tolist()

todel = list()
for idx, m in enumerate(missing):
    if m >= 289:
        todel.append(idx)
todel

[]

In [11]:
# immputation using KNN 20 neigh
from sklearn import neighbors
import numpy as np

In [13]:
# custom version of  https://github.com/bwanglzu/Imputer.py/blob/master/imputer/imputer.py

class Imputer:
    """Imputer class."""
    def __init():
        return

    def _fit(self, X, column, k=10, is_categorical=False):
        """Fit a knn classifier for missing column.
        - Args:
                X(numpy.ndarray): input data
                column(int): column id to be imputed
                k(int): number of nearest neighbors, default 10
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                clf: trained k nearest neighbour classifier
        """
        clf = None
        if not is_categorical:
            clf = neighbors.KNeighborsRegressor(n_neighbors=k, n_jobs=4)
        else:
            clf = neighbors.KNeighborsClassifier(n_neighbors=k, n_jobs=4)
        # use column not null to train the kNN classifier
        missing_idxes = np.where(pd.isnull(X[:, column]))[0]
        if len(missing_idxes) == 0:
            return None
        X_copy = np.delete(X, missing_idxes, 0)
        X_train = np.delete(X_copy, column, 1)
        # if other columns still have missing values fill with mean
        col_mean = None
        if not is_categorical:
            col_mean = np.nanmean(X, 0)
        else:
            col_mean = np.nanmedian(X, 0)
        for col_id in range(0, len(col_mean) - 1):
            col_missing_idxes = np.where(np.isnan(X_train[:, col_id]))[0]
            if len(col_missing_idxes) == 0:
                continue
            else:
                X_train[col_missing_idxes, col_id] = col_mean[col_id]
        y_train = X_copy[:, column]
        # fit classifier
        clf.fit(X_train, y_train)
        return clf

    def _transform(self, X, column, clf, is_categorical):
        """Impute missing values.
        - Args:
                X(numpy.ndarray): input numpy ndarray
                column(int): index of column to be imputed
                clf: pretrained classifier
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                X(pandas.dataframe): imputed dataframe
        """
        missing_idxes = np.where(np.isnan(X[:, column]))[0]
        X_test = X[missing_idxes, :]
        X_test = np.delete(X_test, column, 1)
        # if other columns still have missing values fill with mean
        col_mean = None
        if not is_categorical:
            col_mean = np.nanmean(X, 0)
        else:
            col_mean = np.nanmedian(X, 0)
        # fill missing values in each column with current col_mean
        for col_id in range(0, len(col_mean) - 1):
            col_missing_idxes = np.where(np.isnan(X_test[:, col_id]))[0]
            # if no missing values for current column
            if len(col_missing_idxes) == 0:
                continue
            else:
                X_test[col_missing_idxes, col_id] = col_mean[col_id]
        # predict missing values
        y_test = clf.predict(X_test)
        X[missing_idxes, column] = y_test
        return X

    def knn(self, X, column, k=10, is_categorical=False):
        """Impute missing value with knn.
        - Args:
                X(pandas.dataframe): dataframe
                column(str): column name to be imputed
                k(int): number of nearest neighbors, default 10
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                X_imputed(pandas.dataframe): imputed pandas dataframe
        """
        X, column = self._check_X_y(X, column)
        clf = self._fit(X, column, k, is_categorical)
        if clf is None:
            return X
        else:
            X_imputed = self._transform(X, column, clf, is_categorical)
            return X_imputed

    def _check_X_y(self, X, column):
        """Check input, if pandas.dataframe, transform to numpy array.
        - Args:
                X(ndarray/pandas.dataframe): input instances
                column(str/int): column index or column name
        - Returns:
                X(ndarray): input instances
        """
        column_idx = None
        X = X.select_dtypes(include=[np.number])

        if isinstance(X, pd.core.frame.DataFrame):
            if isinstance(column, str):
                # get index of current column
                column_idx = X.columns.get_loc(column)
            else:
                column_idx = column
            X = X.as_matrix()
        else:
            column_idx = column
        return X, column_idx

In [14]:
imputer = Imputer()

In [15]:
imirna = mirna.copy()
#for c in icnv.columns:
#    missing = icnv[c].isnull().sum()
#    if missing > 0:
#        imputed = imputer.knn(icnv, c, k=20 )
#        icnv[c] = imputed[c]
        
#a = imputer.knn(icnv, "MB-5104",k=20)       


In [18]:
for c in imirna.columns:
    missing = imirna[c].isnull().sum()
    if missing > 0:
#        imputed = imputer.knn(icnv, c, k=20 )
        imirna[c] = imputer.knn(iexp, c, k=20 )
        
#a = imputer.knn(icnv, "MB-5104",k=20)      

In [17]:
sum(mirna.isnull().sum())

0

In [19]:
mirna.to_csv('METABRIC_miRNA_data_implaced.txt',index=False,sep='\t')

In [20]:
!ls

2018-03-02-preprocessing-metabric-CNV-Cleaning.ipynb
2018-03-02-preprocessing-metabric-EXP-Cleaning.ipynb
2018-03-02-preprocessing-metabric-mirna-Cleaning.ipynb
2018-03-2-Merging-with-CNV.ipynb
METABRICCNV_CNV_data.txt
METABRIC_CNV_data.txt
METABRICCNV_Gene_Expression.txt
METABRIC_CNV_implaced_transformed.txt
METABRIC_CNV_implaced.txt
METABRICCNV_Survival.txt
METABRIC_Gene_Expression_data_implaced.txt
METABRIC_Gene_Expression.txt
METABRIC_miRNA_data_implaced.txt
METABRIC_Mirna_Expression.txt
METABRIC_Survival.txt
preprocessing-metabric-CNV.ipynb
preprocessing-metabric.ipynb
Untitled1.ipynb
Untitled2.ipynb
Untitled.ipynb
